In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from imp import reload
import vnn2
reload(vnn2)

<module 'vnn2' from '/home/davidclark/Projects/VectorizedNets/vnn2.py'>

In [85]:
64*8*8

4096

In [131]:
reload(vnn2)
model = nn.Sequential(
    vnn2.Linear(10, 784, 100),
    vnn2.tReLU(10, 100),
    vnn2.Linear(10, 100, 50),
    vnn2.tReLU(10, 50),
    vnn2.Linear(10, 50, 1)
)



In [252]:
reload(vnn2)
model = nn.Sequential(
    vnn2.Conv2d(10, 3, 64, 2, stride=2, padding=0),
    vnn2.ctReLU(10, 64, 8, 8),
    vnn2.AvgPool2d(2),
    vnn2.Flatten(),
    vnn2.Linear(10, 1024, 150),
    vnn2.tReLU(10, 150),
    vnn2.Linear(10, 150, 100),
    vnn2.tReLU(10, 100),
    vnn2.Linear(10, 100, 1)
)



In [253]:
x = torch.randn(128, 10, 3, 16, 16)
#output_error = torch.randn(128, 10)
labels = torch.randint(0, 10, (128,))
targets = torch.eye(10)[labels]

In [254]:
y = model(x)[..., 0]
p = F.softmax(y, dim=1)
y.shape, p.shape
output_error = p - targets

In [255]:
g = torch.ones(128, 1)
for i in list(range(len(model)))[::-1]:
    layer = model[i]
    print(i, layer, g.shape)
    g = layer.custom_backward(g, output_error)


8 Linear() torch.Size([128, 1])
7 tReLU() torch.Size([128, 100])
6 Linear() torch.Size([128, 100])
5 tReLU() torch.Size([128, 150])
4 Linear() torch.Size([128, 150])
3 Flatten() torch.Size([128, 1024])
2 AvgPool2d(
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
) torch.Size([128, 64, 4, 4])
1 ctReLU() torch.Size([128, 64, 8, 8])
0 Conv2d() torch.Size([128, 64, 8, 8])


In [256]:
grads_1 = list(p.grad for (name, p) in model.named_parameters() if name[-2:] != ".t")


In [257]:
for (name, p) in model.named_parameters():
    if name[-2:] != ".t":
        p.grad = None
        p.requires_grad = True
        
grads_2 = list(p.grad for (name, p) in model.named_parameters() if name[-2:] != ".t")


loss_fn = torch.nn.CrossEntropyLoss(reduction="mean")
loss = loss_fn(model(x)[..., 0], labels)
loss.backward()

grads_3 = list(p.grad for (name, p) in model.named_parameters() if name[-2:] != ".t")


In [259]:
for i in range(len(grads_1)):
    g1, g3 = grads_1[i], grads_3[i]
    print(torch.sum(torch.abs(g1 - g3)))

tensor(4.7380e-06)
tensor(4.9067e-07)
tensor(0.0001)
tensor(5.5961e-07)
tensor(9.1092e-06)
tensor(1.9379e-07)
tensor(1.4380e-06)
tensor(1.6764e-08)


In [214]:
model[0].weight.requires_grad

False

In [174]:
x_prime = F.conv_transpose2d(y, weight=W, stride=4, padding=0) #, output_padding=0)
x_prime.shape

torch.Size([128, 3, 32, 32])